In [10]:
import os
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

from pathlib import Path
home = str(Path.home())

In [11]:
def predictions(clf, X_test, scaler=None):
    if not scaler == None:
        X_test   = scaler.transform(X_test)
    y_hat    = clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    return y_hat, y_hat_pr

In [12]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    
    clf = LogisticRegressionCV(max_iter=100, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat, y_hat_pr = predictions(clf, X_test, scaler)
    
    return clf, scaler, y_hat, y_hat_pr

In [13]:
def RF(X_train, y_train, X_test, y_test):
    kwargs = {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 60, 'bootstrap': False}  

    clf = RandomForestClassifier(n_jobs=-1, **kwargs)
    clf.fit(X_train, y_train)
    y_hat, y_hat_pr = predictions(clf, X_test)
    
    return clf, y_hat, y_hat_pr

In [14]:
def prepare_datasets(nor, adv, factor=0.8):
    if len(nor.shape) > 2: 
        nor = nor.reshape((nor.shape[0], -1))
        adv = adv.reshape((adv.shape[0], -1))

    y_nor = np.zeros(nor.shape[0]).astype('int')
    y_adv = np.ones(adv.shape[0]).astype('int')

    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

    X_train = np.concatenate((x_train_n, x_train_a))
    y_train = np.concatenate((y_train_n, y_train_a))

    X_test = np.concatenate((x_test_n, x_test_a))
    y_test = np.concatenate((y_test_n, y_test_a))

    return X_train, y_train, X_test, y_test

In [15]:
factor = 0.8
nr_mean = 3
mean_tables = []
nr_samples=2000

attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
detectors = ['lid', 'multilid']


results = {}

for it in range(nr_mean):
    # it = 2
    random_state = [21, 30, 65][it] # random.randint(100)
    final_table = np.zeros((len(attacks), 2))
    base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
    results[it] = {}

    for dataset in datasets:
        if dataset == 'imagenet':
            k=30
        else:
            k=20

        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue

            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}

            for detector in detectors:
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))[:nr_samples]
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))[:nr_samples]
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))[:nr_samples]
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))[:nr_samples]
                
                assert(nor_fgsm.shape[0] == nr_samples)
                assert(nor_bim.shape[0] == nr_samples)
                assert(nor_pgd.shape[0] == nr_samples)
                assert(nor_aa.shape[0] == nr_samples)
                assert(nor_df.shape[0] == nr_samples)
                assert(nor_cw.shape[0] == nr_samples)

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]

                for source in range(len(nor)):
                    X_train_src, y_train_src, X_test_src, y_test_src = prepare_datasets(nor[source], adv[source], factor)
                    clf_lr, scaler, y_hat_lr, y_hat_pr_lr = LR(X_train_src, y_train_src, X_test_src, y_test_src)
                    clf_rf, y_hat_rf, y_hat_pr_rf = RF(X_train_src, y_train_src, X_test_src, y_test_src)

                    for target in range(len(nor)):
                        if source == target:
                            continue

                        auc_lr = 0;
                        auc_rf = 0
                        f1_lr = 0
                        f1_rf = 0

                        unique_classes = np.unique(y_test_src)
                        if len(unique_classes) < 2:
                            print("ROC AUC score is not defined when there is only one class.")
                            print(f"it:{it}, {dataset}, {model}, {detector}, {source}, {target}")
                        else:
                            X_train_tar, y_train_tar, X_test_tar, y_test_tar = prepare_datasets(nor[target], adv[target], factor)
                            y_test_lr, y_hat_pr_lr = predictions(clf_lr, X_test_tar, scaler)
                            auc_lr = round(100*roc_auc_score(y_test_tar, y_hat_pr_lr), 1)
                            # f1_lr =  round(100*f1_score(y_test_tar, y_hat_lr), 1)
                            acc_lr =  round(100*accuracy_score(y_test_tar, y_test_lr), 1)

                            y_test_rf, y_hat_pr_rf = predictions(clf_rf, X_test_tar)
                            auc_rf = round(100*roc_auc_score(y_test_tar, y_hat_pr_rf), 1)
                            # f1_rf =  round(100*f1_score(y_test_tar, y_hat_rf), 1)
                            acc_rf =  round(100*accuracy_score(y_test_tar, y_test_rf), 1)
                            print(f"it:{it}, {dataset}, {model}, {detector}, {attacks[source]}, {attacks[target]}, {auc_lr}, {acc_lr}")
                            print(f"it:{it}, {dataset}, {model}, {detector}, {attacks[source]}, {attacks[target]}, {auc_rf}, {acc_rf}")

                        if not detector in results[it][dataset][model]:
                            results[it][dataset][model][detector] = {}
                        if not attacks[source] in results[it][dataset][model][detector]: 
                            results[it][dataset][model][detector][attacks[source]] = {}
                        if not attacks[target] in results[it][dataset][model][detector][attacks[source]]: 
                            results[it][dataset][model][detector][attacks[source]][attacks[target]] = {}

                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['auc_lr'] = auc_lr
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['auc_rf'] = auc_rf
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['acc_lr'] = acc_lr
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['acc_rf'] = acc_rf

it:0, cifar10, wrn28-10, lid, fgsm, bim, 75.9, 64.2
it:0, cifar10, wrn28-10, lid, fgsm, bim, 81.3, 71.1
it:0, cifar10, wrn28-10, lid, fgsm, pgd, 77.8, 65.9
it:0, cifar10, wrn28-10, lid, fgsm, pgd, 82.4, 70.0
it:0, cifar10, wrn28-10, lid, fgsm, aa, 86.2, 77.2
it:0, cifar10, wrn28-10, lid, fgsm, aa, 90.0, 81.9
it:0, cifar10, wrn28-10, lid, fgsm, df, 78.3, 66.0
it:0, cifar10, wrn28-10, lid, fgsm, df, 80.1, 68.6
it:0, cifar10, wrn28-10, lid, fgsm, cw, 75.5, 62.9
it:0, cifar10, wrn28-10, lid, fgsm, cw, 77.1, 66.4
it:0, cifar10, wrn28-10, lid, bim, fgsm, 89.4, 81.1
it:0, cifar10, wrn28-10, lid, bim, fgsm, 83.9, 72.6
it:0, cifar10, wrn28-10, lid, bim, pgd, 86.0, 77.8
it:0, cifar10, wrn28-10, lid, bim, pgd, 89.7, 81.9
it:0, cifar10, wrn28-10, lid, bim, aa, 96.8, 88.9
it:0, cifar10, wrn28-10, lid, bim, aa, 95.7, 88.6
it:0, cifar10, wrn28-10, lid, bim, df, 89.0, 81.1
it:0, cifar10, wrn28-10, lid, bim, df, 79.6, 70.0
it:0, cifar10, wrn28-10, lid, bim, cw, 82.6, 73.9
it:0, cifar10, wrn28-10, lid, 

/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

it:0, cifar10, wrn28-10, multilid, fgsm, bim, 86.5, 76.6
it:0, cifar10, wrn28-10, multilid, fgsm, bim, 84.2, 71.6
it:0, cifar10, wrn28-10, multilid, fgsm, pgd, 91.6, 83.8
it:0, cifar10, wrn28-10, multilid, fgsm, pgd, 89.0, 79.2
it:0, cifar10, wrn28-10, multilid, fgsm, aa, 93.5, 88.1
it:0, cifar10, wrn28-10, multilid, fgsm, aa, 94.8, 89.2
it:0, cifar10, wrn28-10, multilid, fgsm, df, 82.4, 74.8
it:0, cifar10, wrn28-10, multilid, fgsm, df, 85.7, 75.6
it:0, cifar10, wrn28-10, multilid, fgsm, cw, 77.9, 68.4
it:0, cifar10, wrn28-10, multilid, fgsm, cw, 81.2, 70.4


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

it:0, cifar10, wrn28-10, multilid, bim, fgsm, 91.9, 83.9
it:0, cifar10, wrn28-10, multilid, bim, fgsm, 86.6, 74.2
it:0, cifar10, wrn28-10, multilid, bim, pgd, 97.1, 91.1
it:0, cifar10, wrn28-10, multilid, bim, pgd, 96.6, 91.4
it:0, cifar10, wrn28-10, multilid, bim, aa, 99.3, 93.2
it:0, cifar10, wrn28-10, multilid, bim, aa, 97.9, 94.2
it:0, cifar10, wrn28-10, multilid, bim, df, 86.5, 79.8
it:0, cifar10, wrn28-10, multilid, bim, df, 79.1, 55.9
it:0, cifar10, wrn28-10, multilid, bim, cw, 79.7, 72.5
it:0, cifar10, wrn28-10, multilid, bim, cw, 76.5, 53.8


...

it:2, imagenet, wrn50-2, multilid, pgd, fgsm, 59.7, 54.1
it:2, imagenet, wrn50-2, multilid, pgd, fgsm, 56.1, 51.9
it:2, imagenet, wrn50-2, multilid, pgd, bim, 94.1, 86.8
it:2, imagenet, wrn50-2, multilid, pgd, bim, 92.5, 84.5
it:2, imagenet, wrn50-2, multilid, pgd, aa, 99.8, 93.4
it:2, imagenet, wrn50-2, multilid, pgd, aa, 99.8, 93.8
it:2, imagenet, wrn50-2, multilid, pgd, df, 54.8, 51.6
it:2, imagenet, wrn50-2, multilid, pgd, df, 53.8, 51.0
it:2, imagenet, wrn50-2, multilid, pgd, cw, 53.0, 51.9
it:2, imagenet, wrn50-2, multilid, pgd, cw, 52.1, 51.5
it:2, imagenet, wrn50-2, multilid, aa, fgsm, 56.9, 50.2
it:2, imagenet, wrn50-2, multilid, aa, fgsm, 50.3, 50.0
it:2, imagenet, wrn50-2, multilid, aa, bim, 91.5, 68.8
it:2, imagenet, wrn50-2, multilid, aa, bim, 85.6, 61.6
it:2, imagenet, wrn50-2, multilid, aa, pgd, 93.8, 70.0
it:2, imagenet, wrn50-2, multilid, aa, pgd, 90.0, 72.5
it:2, imagenet, wrn50-2, multilid, aa, df, 54.3, 50.4
it:2, imagenet, wrn50-2, multilid, aa, df, 52.2, 50.0
it:2

/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

it:2, imagenet, wrn50-2, multilid, df, fgsm, 55.5, 53.4
it:2, imagenet, wrn50-2, multilid, df, fgsm, 55.3, 54.0
it:2, imagenet, wrn50-2, multilid, df, bim, 93.6, 72.4
it:2, imagenet, wrn50-2, multilid, df, bim, 72.9, 66.0
it:2, imagenet, wrn50-2, multilid, df, pgd, 93.6, 75.4
it:2, imagenet, wrn50-2, multilid, df, pgd, 75.8, 67.5
it:2, imagenet, wrn50-2, multilid, df, aa, 99.3, 72.1
it:2, imagenet, wrn50-2, multilid, df, aa, 60.2, 57.0
it:2, imagenet, wrn50-2, multilid, df, cw, 53.7, 53.8
it:2, imagenet, wrn50-2, multilid, df, cw, 53.6, 53.2


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

it:2, imagenet, wrn50-2, multilid, cw, fgsm, 53.8, 51.9
it:2, imagenet, wrn50-2, multilid, cw, fgsm, 53.1, 53.1
it:2, imagenet, wrn50-2, multilid, cw, bim, 91.3, 74.1
it:2, imagenet, wrn50-2, multilid, cw, bim, 73.6, 66.1
it:2, imagenet, wrn50-2, multilid, cw, pgd, 91.2, 75.8
it:2, imagenet, wrn50-2, multilid, cw, pgd, 76.8, 69.9
it:2, imagenet, wrn50-2, multilid, cw, aa, 99.5, 74.1
it:2, imagenet, wrn50-2, multilid, cw, aa, 75.6, 67.9
it:2, imagenet, wrn50-2, multilid, cw, df, 54.8, 53.1
it:2, imagenet, wrn50-2, multilid, cw, df, 52.1, 51.1


# LID

In [16]:
source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# source_attacks = ['fgsm',  'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['lid'][source_attack][target_attack].items()):
                        # print(val)
                        if 'auc_lr' in val[0]:
                            tmp[iter_tar, 0] = val[1]
                        if 'acc_lr' in val[0]:
                            tmp[iter_tar, 1] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean,axis=0)
lid_var = np.var(lid_list_mean,axis=0)

it = 0

for i in range(lid_mean.shape[0]):
    
    attacks_saved = target_attacks.copy()
    
    if i % 5 == 0:
        # print(it)
        source = source_attacks[it]
        it = it + 1
    attacks_saved.remove(source)

    latex_table = ""
    latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[int((i) % 5)].upper()}"+"} & "
    for j in range(lid_mean.shape[1]): 
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        
    latex_table = latex_table[:-2] + "\\\\"
    
    if int((i+1) % 5)==0:
        
        latex_table += " \n\hline"
        latex_table += " \n"
        
        start = i+1-5
        end = i+1
        avg_mean = lid_mean[start:end].mean(axis=1).flatten()
        avg_var  = lid_var[start:end].mean(axis=1).flatten()
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{$\mu$}"
        for k in range(len(avg_mean)):
            latex_table += f" & ${avg_mean[k]:.1f} \\pm {avg_var[k]:.1f}$"
        latex_table = latex_table + "\\\\ \n\hline"
        
        latex_table += "\n\n"
    
    print(latex_table)

\textbf{FGSM} & \textbf{BIM} & $61.4 \pm 107.0$ & $55.1 \pm 41.4$ & $62.0 \pm 124.4$ & $56.8 \pm 41.4$ & $65.4 \pm 216.4$ & $61.5 \pm 123.8$ & $79.0 \pm 1.5$ & $64.6 \pm 3.8$ & $76.2 \pm 0.7$ & $61.8 \pm 0.7$ \\
\textbf{FGSM} & \textbf{PGD} & $77.5 \pm 71.1$ & $60.8 \pm 205.4$ & $87.3 \pm 0.9$ & $78.4 \pm 0.7$ & $95.8 \pm 0.5$ & $87.4 \pm 1.1$ & $65.7 \pm 271.5$ & $60.7 \pm 208.1$ & $59.9 \pm 257.7$ & $57.9 \pm 128.5$ \\
\textbf{FGSM} & \textbf{AA} & $78.3 \pm 71.5$ & $60.8 \pm 227.6$ & $86.8 \pm 0.4$ & $79.5 \pm 1.5$ & $96.5 \pm 0.3$ & $88.7 \pm 0.8$ & $66.5 \pm 326.9$ & $61.8 \pm 273.0$ & $61.8 \pm 321.8$ & $59.4 \pm 176.1$ \\
\textbf{FGSM} & \textbf{DF} & $78.5 \pm 52.7$ & $60.2 \pm 193.4$ & $86.5 \pm 0.1$ & $74.0 \pm 0.5$ & $87.9 \pm 1.7$ & $74.8 \pm 4.4$ & $66.5 \pm 299.4$ & $60.6 \pm 210.8$ & $61.8 \pm 292.8$ & $58.4 \pm 139.4$ \\
\textbf{FGSM} & \textbf{CW} & $87.9 \pm 1.2$ & $79.8 \pm 0.2$ & $66.5 \pm 97.5$ & $55.0 \pm 50.7$ & $66.2 \pm 146.3$ & $55.4 \pm 58.3$ & $70.8 \pm 299.

In [17]:
source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# source_attacks = ['fgsm',  'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['lid'][source_attack][target_attack].items()):
                        # print(val)
                        if 'auc_rf' in val[0]:
                            tmp[iter_tar, 0] = val[1]
                        if 'acc_rf' in val[0]:
                            tmp[iter_tar, 1] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean,axis=0)
lid_var = np.var(lid_list_mean,axis=0)

it = 0

for i in range(lid_mean.shape[0]):
    
    attacks_saved = target_attacks.copy()
    
    if i % 5 == 0:
        # print(it)
        source = source_attacks[it]
        it = it + 1
    attacks_saved.remove(source)

    latex_table = ""
    latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[int((i) % 5)].upper()}"+"} & "
    for j in range(lid_mean.shape[1]): 
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        
    latex_table = latex_table[:-2] + "\\\\"
    
    if int((i+1) % 5)==0:
        
        latex_table += " \n\hline"
        latex_table += " \n"
        
        start = i+1-5
        end = i+1
        avg_mean = lid_mean[start:end].mean(axis=1).flatten()
        avg_var  = lid_var[start:end].mean(axis=1).flatten()
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{$\mu$}"
        for k in range(len(avg_mean)):
            latex_table += f" & ${avg_mean[k]:.1f} \\pm {avg_var[k]:.1f}$"
        latex_table += "\\\\ \n\hline"
        latex_table += "\n"
    
    print(latex_table)

\textbf{FGSM} & \textbf{BIM} & $62.2 \pm 184.7$ & $56.9 \pm 101.1$ & $59.9 \pm 252.4$ & $54.9 \pm 113.6$ & $66.5 \pm 277.9$ & $61.0 \pm 225.5$ & $79.5 \pm 2.3$ & $66.8 \pm 1.9$ & $75.6 \pm 1.4$ & $63.9 \pm 7.1$ \\
\textbf{FGSM} & \textbf{PGD} & $53.5 \pm 469.9$ & $53.0 \pm 192.1$ & $90.0 \pm 1.0$ & $82.7 \pm 2.2$ & $94.9 \pm 0.7$ & $88.2 \pm 1.7$ & $64.0 \pm 121.7$ & $58.9 \pm 62.0$ & $58.7 \pm 113.1$ & $55.5 \pm 52.7$ \\
\textbf{FGSM} & \textbf{AA} & $54.7 \pm 584.1$ & $55.1 \pm 341.3$ & $87.4 \pm 0.2$ & $79.8 \pm 1.0$ & $95.7 \pm 1.0$ & $89.1 \pm 2.5$ & $65.6 \pm 213.6$ & $61.2 \pm 127.0$ & $60.0 \pm 239.9$ & $57.5 \pm 106.2$ \\
\textbf{FGSM} & \textbf{DF} & $56.8 \pm 529.7$ & $58.4 \pm 220.4$ & $86.3 \pm 0.9$ & $72.9 \pm 0.2$ & $89.0 \pm 4.7$ & $77.2 \pm 5.1$ & $68.6 \pm 203.0$ & $62.0 \pm 115.4$ & $62.8 \pm 239.2$ & $57.0 \pm 86.7$ \\
\textbf{FGSM} & \textbf{CW} & $86.4 \pm 1.6$ & $77.6 \pm 0.4$ & $64.5 \pm 98.8$ & $54.5 \pm 41.1$ & $66.9 \pm 104.8$ & $56.5 \pm 83.6$ & $74.3 \pm 21

# multiLID

In [18]:
source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# source_attacks = ['fgsm',  'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['multilid'][source_attack][target_attack].items()):
                        # print(val)
                        if 'auc_lr' in val[0]:
                            tmp[iter_tar, 0] = val[1]
                        if 'acc_lr' in val[0]:
                            tmp[iter_tar, 1] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean,axis=0)
lid_var = np.var(lid_list_mean,axis=0)

it = 0

for i in range(lid_mean.shape[0]):
    
    attacks_saved = target_attacks.copy()
    
    if i % 5 == 0:
        # print(it)
        source = source_attacks[it]
        it = it + 1
    attacks_saved.remove(source)

    latex_table = ""
    latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[int((i) % 5)].upper()}"+"} & "
    for j in range(lid_mean.shape[1]): 
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        
    latex_table = latex_table[:-2] + "\\\\"
    
    if int((i+1) % 5)==0:
        
        latex_table += " \n\hline"
        latex_table += " \n"
        
        start = i+1-5
        end = i+1
        avg_mean = lid_mean[start:end].mean(axis=1).flatten()
        avg_var  = lid_var[start:end].mean(axis=1).flatten()
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{$\mu$}"
        for k in range(len(avg_mean)):
            latex_table += f" & ${avg_mean[k]:.1f} \\pm {avg_var[k]:.1f}$"
        latex_table = latex_table + "\\\\ \n\hline"
        
        latex_table += "\n\n"
    
    print(latex_table)

\textbf{FGSM} & \textbf{BIM} & $68.3 \pm 165.1$ & $59.4 \pm 148.5$ & $90.6 \pm 1.7$ & $76.7 \pm 27.8$ & $85.6 \pm 162.8$ & $76.4 \pm 222.1$ & $81.0 \pm 3.7$ & $70.0 \pm 11.6$ & $77.0 \pm 0.6$ & $65.5 \pm 4.8$ \\
\textbf{FGSM} & \textbf{PGD} & $42.2 \pm 1237.7$ & $60.1 \pm 284.0$ & $45.2 \pm 1348.5$ & $64.0 \pm 367.2$ & $69.2 \pm 1822.1$ & $79.2 \pm 426.6$ & $31.9 \pm 1489.1$ & $57.9 \pm 241.3$ & $29.7 \pm 1250.3$ & $53.3 \pm 186.0$ \\
\textbf{FGSM} & \textbf{AA} & $93.5 \pm 2.5$ & $86.3 \pm 8.5$ & $50.6 \pm 971.1$ & $63.3 \pm 320.1$ & $70.9 \pm 1583.1$ & $80.4 \pm 456.0$ & $86.8 \pm 1.0$ & $77.6 \pm 0.3$ & $82.5 \pm 5.9$ & $73.9 \pm 1.9$ \\
\textbf{FGSM} & \textbf{DF} & $69.8 \pm 1047.7$ & $70.8 \pm 251.3$ & $71.1 \pm 1009.7$ & $68.9 \pm 172.4$ & $73.3 \pm 1014.3$ & $73.6 \pm 266.8$ & $60.4 \pm 1429.3$ & $64.9 \pm 173.4$ & $57.3 \pm 1316.4$ & $61.3 \pm 125.7$ \\
\textbf{FGSM} & \textbf{CW} & $86.8 \pm 1.1$ & $79.2 \pm 0.3$ & $55.9 \pm 568.7$ & $53.8 \pm 22.1$ & $87.3 \pm 5.0$ & $64.6 \

In [19]:
source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# source_attacks = ['fgsm',  'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['multilid'][source_attack][target_attack].items()):
                        # print(val)
                        if 'auc_rf' in val[0]:
                            tmp[iter_tar, 0] = val[1]
                        if 'acc_rf' in val[0]:
                            tmp[iter_tar, 1] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean,axis=0)
lid_var = np.var(lid_list_mean,axis=0)

it = 0

for i in range(lid_mean.shape[0]):
    
    attacks_saved = target_attacks.copy()
    
    if i % 5 == 0:
        # print(it)
        source = source_attacks[it]
        it = it + 1
    attacks_saved.remove(source)

    latex_table = ""
    latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[int((i) % 5)].upper()}"+"} & "
    for j in range(lid_mean.shape[1]): 
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        
    latex_table = latex_table[:-2] + "\\\\"
    
    if int((i+1) % 5)==0:
        
        latex_table += " \n\hline"
        latex_table += " \n"
        
        start = i+1-5
        end = i+1
        avg_mean = lid_mean[start:end].mean(axis=1).flatten()
        avg_var  = lid_var[start:end].mean(axis=1).flatten()
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{$\mu$}"
        for k in range(len(avg_mean)):
            latex_table += f" & ${avg_mean[k]:.1f} \\pm {avg_var[k]:.1f}$"
        latex_table += "\\\\ \n\hline"
        latex_table += "\n"
    
    print(latex_table)

\textbf{FGSM} & \textbf{BIM} & $62.2 \pm 242.3$ & $56.8 \pm 109.6$ & $87.6 \pm 1.1$ & $71.4 \pm 31.0$ & $83.0 \pm 317.1$ & $77.3 \pm 251.9$ & $84.9 \pm 3.4$ & $72.9 \pm 3.8$ & $81.9 \pm 1.3$ & $68.3 \pm 4.2$ \\
\textbf{FGSM} & \textbf{PGD} & $60.9 \pm 338.8$ & $56.9 \pm 151.5$ & $68.4 \pm 397.1$ & $66.1 \pm 319.7$ & $83.7 \pm 358.7$ & $80.9 \pm 286.8$ & $63.2 \pm 127.3$ & $54.7 \pm 1.0$ & $59.0 \pm 154.5$ & $51.1 \pm 4.3$ \\
\textbf{FGSM} & \textbf{AA} & $85.0 \pm 0.1$ & $73.1 \pm 5.8$ & $70.4 \pm 296.7$ & $66.7 \pm 231.3$ & $86.3 \pm 322.6$ & $83.6 \pm 315.3$ & $83.6 \pm 2.0$ & $56.7 \pm 0.5$ & $83.8 \pm 0.3$ & $55.8 \pm 0.8$ \\
\textbf{FGSM} & \textbf{DF} & $78.4 \pm 250.9$ & $69.4 \pm 102.8$ & $77.4 \pm 130.7$ & $61.6 \pm 0.7$ & $80.1 \pm 292.4$ & $68.3 \pm 81.1$ & $80.4 \pm 285.6$ & $62.7 \pm 57.0$ & $78.3 \pm 394.3$ & $61.6 \pm 58.9$ \\
\textbf{FGSM} & \textbf{CW} & $86.3 \pm 0.9$ & $78.6 \pm 0.8$ & $59.1 \pm 188.9$ & $49.8 \pm 0.1$ & $83.9 \pm 4.4$ & $55.1 \pm 1.2$ & $80.9 \pm 34